# Capas

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfkl
from scipy.signal import stft, istft 
import numpy as np
import keras


class Spectrogram(tfkl.Layer):
    def __init__(self,win_size,hop_size,fft_size=None,calculate='magnitude',window=tf.signal.hann_window,pad_end=False,name=None, trainable=False):
        super(Spectrogram, self).__init__(name=name)

        self.stft_args = {'ws': win_size,
                  'hs': hop_size,
                  'ffts': fft_size,
                  'win': window,
                  'pad': pad_end,
                  'calculate': calculate}

    def call(self,x):
        stft = tf.signal.stft(
                signals=x,
                frame_length=self.stft_args['ws'],
                frame_step=self.stft_args['hs'],
                fft_length=self.stft_args['ffts'],
                window_fn=self.stft_args['win'],
                pad_end=self.stft_args['pad'])

        calculate = self.stft_args['calculate']
        if calculate == 'magnitude':
            return tf.abs(stft)
        elif calculate == 'complex':
            return stft
        elif calculate == 'phase':
            return tf.math.angle(stft)
        else:
            raise Exception("{} not recognized as calculate parameter".format(calculate))

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'win_size': self.stft_args['ws'],
            'hop_size': self.stft_args['hs'],
            'fft_size': self.stft_args['ffts'],
            'calculate': self.stft_args['calculate'],
            'window': self.stft_args['win'],
            'pad_end': self.stft_args['pad']
        })
        return config

class SoftMask(tfkl.Layer):
    def __init__(self,name=None):
        super(SoftMask,self).__init__(name=name)

    def call(self,x):
        sources = x[0]
        to_mask = x[1]
        total_sources = tf.expand_dims(tf.reduce_sum(sources,axis=1),axis=1)
        mask = sources/(total_sources+1e-9)
        to_mask = tf.expand_dims(to_mask,axis=1)
        return mask*to_mask

class TranslateRange(tfkl.Layer):
    def __init__(self,name=None,trainable=False,original_range=None,target_range=None):
        super(TranslateRange,self).__init__(name=name)
        self.original_range = original_range
        self.target_range = target_range

    def call(self,x):
        original = self.original_range[1] - self.original_range[0]
        target = self.target_range[1] - self.target_range[0]
        
        #centro_o = self.original_range[1]-self.original_range[0]
        #centro_t = self.target_range[1]-self.target_range[0]
        #offset = centro_t - centro_o
        #x_range = self.original_range[1] - self.original_range[0]
        #y_range = self.target_range[1] - self.target_range[0]

        #return y_range*(x + offset)/x_range
        return (((x - self.original_range[0]) * target) / original) + self.target_range[0]
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'original_range': self.original_range,
            'target_range': self.target_range,
        })
        return config

class Normalize(tfkl.Layer):    
    def __init__(self,name=None,trainable=False,original_range=None,target_range=None):
        super(Normalize,self).__init__(name=name)
        self.original_range = original_range
        self.target_range = target_range

    def call(self,x):
        factor = tf.math.reduce_max(tf.math.abs(x))
        return tf.math.scalar_mul(1/factor, x) 
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'original_range': self.original_range,
            'target_range': self.target_range,
        })
        return config

class Normalize_pos(tfkl.Layer):    
    def __init__(self,name=None,trainable=False,original_range=None,target_range=None):
        super(Normalize_pos,self).__init__(name=name)
        self.original_range = original_range
        self.target_range = target_range

    def call(self,x):
        factor = tf.math.reduce_max(tf.math.abs(x))
        salida = tf.math.scalar_mul(1/(2*factor), x) 
        return  salida + 1
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'original_range': self.original_range,
            'target_range': self.target_range,
        })
        return config



class MSE(tfkl.Layer):
    #x[0]: predicted
    #x[1]: original
    def __init__(self,name=None,trainable=False,lnorm=2,offset=1e-9,normalize=False):
        super(MSE,self).__init__(name=name)
        self.offset = offset
        self.normalize = normalize
        self.lnorm = lnorm
    
    def call(self,x):
        mse_error = tf.abs(x[0] - x[1])**self.lnorm
        if self.normalize:
            mse_error = mse_error/(self.offset + tf.abs(x[1])**self.lnorm)
        return mse_error
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'offset': self.offset,
            'normalize': self.normalize,
            'lnorm' : self.lnorm
        })
        return config

# Modelo a utilizar
Pruebo utilizando un AE común pero calculando una máscara

In [3]:
"""Modelo de red utilizado"""
import sys, os


def autoencoder():
    tf.keras.backend.clear_session()
    eps = np.finfo(float).eps

    ruido = tfkl.Input((256,256,1), name = 'Entrada_ruido')
    clean = tfkl.Input((256,256,1), name = 'Entrada_clean')


    #ENCODER
    enc = tfkl.Conv2D(64, kernel_size=(5,5), strides=2, padding='SAME', input_shape=(256,256,1), name='CONV1')(ruido)
    enc_1 = tfkl.LeakyReLU(alpha = 0.2, name = 'ACT1')(enc)

    enc_2 = tfkl.Conv2D(128, kernel_size=(5,5), strides=2, padding='SAME', name='CONV2')(enc_1)
    enc_2 = tfkl.BatchNormalization(name='BATCH2')(enc_2)
    enc_2 = tfkl.LeakyReLU(alpha = 0.2, name='ACT2')(enc_2)

    enc_3 = tfkl.Conv2D(256, kernel_size=(5,5), strides=2, padding='SAME', name='CONV3')(enc_2)
    enc_3 = tfkl.BatchNormalization(name='BATCH3')(enc_3)
    enc_3 = tfkl.LeakyReLU(alpha = 0.2, name='ACT3')(enc_3)

    enc_4 = tfkl.Conv2D(512, kernel_size=(5,5), strides=2, padding='SAME', name='CONV4')(enc_3)
    enc_4 = tfkl.BatchNormalization(name='BATCH4')(enc_4)
    enc_4 = tfkl.LeakyReLU(alpha = 0.2, name='ACT4')(enc_4)

    enc_5 = tfkl.Conv2D(512, kernel_size=(5,5), strides=2, padding='SAME', name='CONV5')(enc_4)
    enc_5 = tfkl.BatchNormalization(name='BATCH5')(enc_5)
    enc_5 = tfkl.LeakyReLU(alpha = 0.2, name='ACT5')(enc_5)

    enc_6 = tfkl.Conv2D(512, kernel_size=(5,5), strides=2, padding='SAME', name='CONV6')(enc_5)
    enc_6 = tfkl.BatchNormalization(name = 'BATCH6')(enc_6)
    enc_6 = tfkl.LeakyReLU(alpha = 0.2, name='ACT6')(enc_6)

    enc_7 = tfkl.Conv2D(512, kernel_size=(5,5), strides=2, padding='SAME', name='CONV7')(enc_6)
    enc_7 = tfkl.BatchNormalization(name = 'BATCH7')(enc_7)
    enc_7 = tfkl.LeakyReLU(alpha = 0.2, name='ACT7')(enc_7)

    enc_8 = tfkl.Conv2D(512, kernel_size=(5,5), strides=2, padding='SAME', name='CONV8')(enc_7)
    enc_8 = tfkl.BatchNormalization(name = 'BATCH8')(enc_8)
    enc_8 = tfkl.ReLU()(enc_8)


    #DECODER
    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(enc_8)
    dec = tfkl.Conv2D(512, kernel_size=(5,5), strides=1, padding='SAME', name='CONV9')(dec)
    #dec = tfkl.Conv2DTranspose(256, kernel_size=(4,4), strides=2, padding='SAME', name='CONV9')(enc_8)
    dec = tfkl.BatchNormalization(name = 'BATCH9')(dec)
    dec = tfkl.Dropout(rate = 0.5)(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_7])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec)
    dec = tfkl.Conv2D(512, kernel_size=(5,5), strides=1, padding='SAME', name='CONV10')(dec)
    #dec = tfkl.Conv2DTranspose(256, kernel_size=(4,4), strides=2, padding='SAME', name='CONV10')(dec)
    dec = tfkl.BatchNormalization(name = 'BATCH10')(dec)
    dec = tfkl.Dropout(rate = 0.5)(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_6])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec)
    dec = tfkl.Conv2D(512, kernel_size=(5,5), strides=1, padding='SAME', name='CONV11')(dec)
    #dec = tfkl.Conv2DTranspose(256, kernel_size=(4,4), strides=2, padding='SAME', name='CONV11')(dec)
    dec = tfkl.BatchNormalization(name = 'BATCH11')(dec)
    dec = tfkl.Dropout(rate = 0.5)(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_5])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec)
    dec = tfkl.Conv2D(512, kernel_size=(5,5), strides=1, padding='SAME', name='CONV12')(dec)
    #dec = tfkl.Conv2DTranspose(256, kernel_size=(4,4), strides=2, padding='SAME', name='CONV12')(dec)
    dec = tfkl.BatchNormalization(name = 'BATCH12')(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_4])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec) #PROVISORIO
    dec = tfkl.Conv2D(256, kernel_size=(5,5), strides=1, padding='SAME', name='CONV13')(dec)
    #dec = tfkl.Conv2DTranspose(128, kernel_size=(4,4), strides=2, padding='SAME', name='CONV13')(dec)
    dec = tfkl.BatchNormalization(name = 'BATCH13')(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_3])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec)
    dec = tfkl.Conv2D(128, kernel_size=(5,5), strides=1, padding='SAME', name='CONV14')(dec)
    #dec = tfkl.Conv2DTranspose(64, kernel_size=(4,4), strides=2, padding='SAME', name='CONV14')(dec)
    dec = tfkl.BatchNormalization(name = 'BATCH14')(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_2])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec)
    dec = tfkl.Conv2D(64, kernel_size=(5,5), strides=1, padding='SAME', name='CONV15')(dec)
    #dec = tfkl.Conv2DTranspose(32, kernel_size=(4,4), strides=2, padding='SAME', name='CONV15')(dec)
    dec = tfkl.BatchNormalization(name = 'BATCH15')(dec)
    dec = tfkl.ReLU()(dec)
    dec = tfkl.Concatenate(axis=-1)([dec, enc_1])

    dec = tfkl.UpSampling2D(size=(2,2), interpolation = 'nearest')(dec)
    dec = tfkl.Conv2D(1, kernel_size=(5,5), strides=1, padding='SAME', name='CONV16')(dec)
    #dec = tfkl.ReLU(max_value=1.0, name = 'SALIDA_DEL_DECODER')(dec)
    dec = tfkl.Activation('sigmoid', name = 'SALIDA_DEL_DECODER')(dec)
    #dec = tfkl.Conv2DTranspose(1, kernel_size=(4,4), strides=2, padding='SAME', activation='tanh', name='CONV16')(dec)

    clean_predict = tfkl.multiply([dec, ruido], name = 'CLEAN_PREDICT')
    clean_predict = tf.pad(clean_predict, ((0,0),(0,0),(0,0),(0,0)), mode='CONSTANT', constant_values=0)

    modelo = tf.keras.Model(inputs=[ruido], outputs=[clean_predict])
    
    modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError()) 

    return modelo 

def mean_loss(y_true, y_pred):
    """
    Custom loss. El error cuadratico ya se calcula en la net, solo lo reduzco a un escalar
    """
    return tf.reduce_mean(y_pred)

# Cargo la data

In [9]:
#Cargo las stft con ruido que voy a usar para entrenar.
X_train = np.load('/content/drive/My Drive/Denoising 6 - Labo de acústica/audios_noise_train_Drive.npy')

for i in range(len(X_train)):
  previa = X_train[i].split('\\')
  X_train[i] = previa[0] + '/' + previa[1]

#Info de los audios
print(len(X_train))
print(X_train[0])
print(X_train[-1])


22000
49/1-common_voice_es_19745638_2_SNR-5.npy
45/1-common_voice_es_19691561_2_SNR-5.npy


In [10]:
#Cargo los audios limpios que voy a usar para entrenar. 
y_train = np.load('/content/drive/My Drive/Denoising 6 - Labo de acústica/audios_clean_train_Drive.npy')

for i in range(len(y_train)):
  previa = y_train[i].split('\\')
  y_train[i] = previa[0] + '/' + previa[1]

#Info de los audios
print(len(y_train))
print(y_train[0])
print(y_train[-1])

22000
49/1-common_voice_es_19745638_2.npy
45/1-common_voice_es_19691561_2.npy


In [11]:
#Función para cargar audios
def load_fsdd(path, files):
    audios = []
    for file_name in files:
        spectrogram = np.load(path + file_name) # (n_bins, n_frames, 1)
        audios.append(spectrogram)
    audios = np.array(audios)
    audios = audios[..., np.newaxis] # -> (N°Audios, 256, 256, 1)
    return audios


In [12]:
dir = os.getcwd() #Directorio actual
print(dir)

/content


In [13]:
#Cargo los audios
print('Cargando audios...')

dir = os.getcwd()

SPECTROGRAMS_NOISE = dir + "/Drive/stft_ruido_train/"
SPECTROGRAMS_CLEAN = dir + "/Drive/stft_train/"

X_train = load_fsdd(SPECTROGRAMS_NOISE ,X_train)
y_train = load_fsdd(SPECTROGRAMS_CLEAN ,y_train)

print(f'Se cargaron {len(X_train)} espectrogramas de X_train')
print(f'Se cargaron {len(y_train)} espectrogramas de y_train')
print('Audios cargados!!! A entrenar mi rey')

Cargando audios...
Se cargaron 22000 espectrogramas de X_train
Se cargaron 22000 espectrogramas de y_train
Audios cargados!!! A entrenar mi rey


In [14]:
X_train[0].shape #Shape de los espectrogramas

(256, 256, 1)

#Entrenando a la bestia

In [15]:
!nvidia-smi

Thu Jun 17 06:09:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:

#Data generators
#from model.data_loader import build_generators
#params = {'path':'/mnt/datasets/npy_data/experimento_1/', 'batch_size' : 16, 'dim' : (256, 257)}
#training_generator, validation_generator = build_generators(params)

#defino el modelo
modelo = autoencoder()

#cbks = [tf.keras.callbacks.EarlyStopping(monitor='loss',restore_best_weights=True, patience=2),
#        tf.keras.callbacks.ModelCheckpoint('/home/martin/Documents/tesis/src/model/ckpts/weights.{epoch:02d}-{loss:.3f}.hdf5'),
#        tf.keras.callbacks.TensorBoard(log_dir='tb_logs',profile_batch=0, update_freq='batch', histogram_freq=1)]

modelo.summary()
#Entreno

LEARNING_RATE = 0.0005
BATCH_SIZE = 16
EPOCHS = 40

#modelo.load_weights('/content/drive/My Drive/Denoising 6 - Labo de acústica/Modelo/weights.hdf5')

history = modelo.fit(X_train,
                     y_train,
                     batch_size=BATCH_SIZE,
                     epochs=EPOCHS)

#history = modelo.fit(training_generator, validation_data =  validation_generator, use_multiprocessing = True, workers=12, epochs=10)
modelo.save_weights('/content/drive/My Drive/Denoising 6 - Labo de acústica/Modelo/weights_40.hdf5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Entrada_ruido (InputLayer)      [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
CONV1 (Conv2D)                  (None, 128, 128, 64) 1664        Entrada_ruido[0][0]              
__________________________________________________________________________________________________
ACT1 (LeakyReLU)                (None, 128, 128, 64) 0           CONV1[0][0]                      
__________________________________________________________________________________________________
CONV2 (Conv2D)                  (None, 64, 64, 128)  204928      ACT1[0][0]                       
______________________________________________________________________________________________

# Genero audios a partir de la red

In [4]:
#Funciones para normalizar y desnormalizar

class MinMaxNormaliser:
    """MinMaxNormaliser applies min max normalisation to an array."""

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def normalise(self, array):
        norm_array = (array - array.min()) / (array.max() - array.min())
        norm_array = norm_array * (self.max - self.min) + self.min
        return norm_array

    def denormalise(self, norm_array, original_min, original_max):
        array = (norm_array - self.min) / (self.max - self.min)
        array = array * (original_max - original_min) + original_min
        return array

In [5]:
class SoundGenerator:
    ###SoundGenerator is responsible for generating audios from
    #spectrograms.
    

    def __init__(self, AE, hop_length):
        self.AE = AE
        self.hop_length = hop_length
        self._min_max_normaliser = MinMaxNormaliser(0, 1)

    def generate(self, spectrograms, min_max_values):
        generated_spectrograms = self.AE.predict(spectrograms)
        signals = self.convert_spectrograms_to_audio(generated_spectrograms, min_max_values)
        return signals

    def convert_spectrograms_to_audio(self, spectrograms, min_max_values):
        signals = []
        for spectrogram, min_max_value in zip(spectrograms, min_max_values):
            # reshape the log spectrogram
            log_spectrogram = spectrogram[:, :, 0]
            # apply denormalisation
            denorm_log_spec = self._min_max_normaliser.denormalise(
                log_spectrogram, min_max_value["min"], min_max_value["max"])
            # log spectrogram -> spectrogram
            spec = librosa.db_to_amplitude(denorm_log_spec)
            # apply Griffin-Lim
            signal = librosa.griffinlim(S=spec, hop_length=self.hop_length)
            # append signal to "signals"
            signals.append(signal)
        return signals

In [35]:
def load_fsdd(spectrograms_path):
    x_train = []
    file_paths = []
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames, 1)
            x_train.append(spectrogram)
            file_paths.append(file_path)
    x_train = np.array(x_train)
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)
    return x_train, file_paths


def select_spectrograms(spectrograms,
                        file_paths,
                        min_max_values,
                        num_spectrograms=42):
    
    file_paths_min_max = []
    for file in file_paths:
      #file_paths_min_max.append('/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/8_stft_ruido_train/'+file.split('/')[-1])
      file_paths_min_max.append('/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/10_stft_ruido_test/'+file.split('/')[-1])

    sampled_indexes = np.random.choice(range(len(spectrograms)), num_spectrograms)
    sampled_spectrogrmas = spectrograms[sampled_indexes]
    file_paths = [file_paths[index] for index in sampled_indexes]
    sampled_min_max_values = [min_max_values[file_path] for file_path in file_paths_min_max]
    print(file_paths)
    print(sampled_min_max_values)
    return sampled_spectrogrmas, sampled_min_max_values, file_paths


def save_signals(signals, save_dir, file_paths, sample_rate=22050):
  for i, signal in enumerate(signals):
    signal = signal/max(signal)
    save_path = os.path.join(save_dir, (file_paths[i].split('/')[-1]).split('.npy')[0] + ".wav")
    sf.write(save_path, signal, sample_rate)

In [34]:
import pickle
import librosa
import soundfile as sf

os.chdir('C:/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo')
dir = os.getcwd()

HOP_LENGTH = 256
SAVE_DIR_ORIGINAL = "11_Audios_para_el_test/original/"
SAVE_DIR_GENERATED = "11_Audios_para_el_test/generated_20/"
MIN_MAX_VALUES_PATH_noise = "min_max_values_final.pkl"



SPECTROGRAMS_NOISE = "stft_ruido_final/"

#X_train = load_fsdd(SPECTROGRAMS_NOISE ,X_train)
#y_train = load_fsdd(SPECTROGRAMS_CLEAN ,y_train)

# initialise sound generator
modelo = autoencoder()
modelo.load_weights('Modelos/weights_20.hdf5')
#vae = VAE.load("model_4-reconstruction_loss_weight_600000-8351_audios")
sound_generator = SoundGenerator(modelo, HOP_LENGTH)

# load spectrograms + min max values
with open(MIN_MAX_VALUES_PATH_noise, "rb") as f:
    min_max_values_noise = pickle.load(f)

specs_noise, file_paths_noise = load_fsdd(SPECTROGRAMS_NOISE)

# sample spectrograms + min max values
sampled_specs, sampled_min_max_values, name_of_audios = select_spectrograms(specs_noise, file_paths_noise, min_max_values_noise, num_spectrograms=42)

# generate audio for sampled spectrograms
signals = sound_generator.generate(sampled_specs,
                                   sampled_min_max_values)

# convert spectrogram samples to audio
original_signals = sound_generator.convert_spectrograms_to_audio(sampled_specs, sampled_min_max_values)

# save audio signals
save_signals(signals, SAVE_DIR_GENERATED, name_of_audios)
save_signals(original_signals, SAVE_DIR_ORIGINAL, name_of_audios)

['stft_ruido_final/common_voice_es_18425698_SNR-0.npy', 'stft_ruido_final/common_voice_es_18868816_SNR-10.npy', 'stft_ruido_final/common_voice_es_19668308_SNR-15.npy', 'stft_ruido_final/common_voice_es_18690582_SNR-5.npy', 'stft_ruido_final/common_voice_es_19720169_SNR-15.npy', 'stft_ruido_final/common_voice_es_18721402_SNR-5.npy', 'stft_ruido_final/common_voice_es_18510484_SNR-5.npy', 'stft_ruido_final/common_voice_es_18571802_SNR-5.npy', 'stft_ruido_final/common_voice_es_18425698_SNR-0.npy', 'stft_ruido_final/common_voice_es_18562961_SNR-5.npy', 'stft_ruido_final/common_voice_es_18309707_SNR-0.npy', 'stft_ruido_final/common_voice_es_18663511_SNR-5.npy', 'stft_ruido_final/common_voice_es_20343036_SNR-15.npy', 'stft_ruido_final/common_voice_es_19624317_SNR-15.npy', 'stft_ruido_final/common_voice_es_19666690_SNR-15.npy', 'stft_ruido_final/common_voice_es_18906236_SNR-10.npy', 'stft_ruido_final/common_voice_es_18690582_SNR-5.npy', 'stft_ruido_final/common_voice_es_18485809_SNR-0.npy', 's

In [24]:
keys = []
for key in min_max_values_noise.keys():
  keys.append(key)
print(keys[0])

/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/10_stft_ruido_test/common_voice_es_18309707_SNR-0.npy


In [30]:
os.getcwd()

'C:\\Users\\maxia\\Desktop\\Facultad\\Labo de acústica\\Codigo'